<a href="https://colab.research.google.com/github/S-Delowar/PlantVision-TF-TL/blob/main/Convert_Keras_Model_to_TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
KERAS_MODEL_PATH = "/content/drive/MyDrive/TensorFlow_Projects/PlantVision_Project/model_checkpoints/best_102_flowes_model_fine_tuned_2.keras"
TFLITE_MODEL_PATH = "/content/drive/MyDrive/TensorFlow_Projects/PlantVision_Project/model_checkpoints/best_102_flowes_model_fine_tuned_2.tflite"

In [3]:
import tensorflow as tf

In [4]:
# Load keras model
keras_model = tf.keras.models.load_model(KERAS_MODEL_PATH)

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
# Converting from keras to tflite formate
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]

tflite_model = converter.convert()

Saved artifact at '/tmp/tmpg8nn63ta'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300, 300, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 102), dtype=tf.float32, name=None)
Captures:
  132065412150096: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float16, name=None)
  132065412149520: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float16, name=None)
  132066093971408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132066077738000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132066077738576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132066093972176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132066093973328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132066077738384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132066077739920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  132066077740880: TensorSpec(shape=(), dtype=tf.resource, name=Non

In [6]:
# Save tflite model
with open(TFLITE_MODEL_PATH, 'wb') as f:
        f.write(tflite_model)
print(f"Model saved to:\n{TFLITE_MODEL_PATH}")

Model saved to:
/content/drive/MyDrive/TensorFlow_Projects/PlantVision_Project/model_checkpoints/best_102_flowes_model_fine_tuned_2.tflite


### Check prediction of TFLite model

In [7]:
image_path = "/content/drive/MyDrive/TensorFlow_Projects/PlantVision_Project/custom_images/bromelia2.jpeg"

In [8]:
# Preprocessing the image

image_size = (300, 300)

img_bytes = tf.io.read_file(image_path)
img_arr = tf.image.decode_image(img_bytes, channels=3)
img_arr = tf.image.resize(img_arr, image_size)

input_img_arr = tf.expand_dims(img_arr, axis=0)

In [9]:
input_img_arr

<tf.Tensor: shape=(1, 300, 300, 3), dtype=float32, numpy=
array([[[[209.      , 212.      , 159.      ],
         [196.5     , 199.5     , 146.5     ],
         [184.875   , 186.75    , 134.875   ],
         ...,
         [105.25    , 131.875   ,  57.125   ],
         [104.5     , 130.375   ,  50.75    ],
         [112.      , 136.      ,  52.      ]],

        [[210.25    , 213.25    , 157.75    ],
         [198.92188 , 201.92188 , 147.20312 ],
         [187.29688 , 189.875   , 136.04688 ],
         ...,
         [107.515625, 132.96875 ,  59.390625],
         [106.765625, 131.23438 ,  53.71875 ],
         [110.75    , 134.125   ,  52.625   ]],

        [[190.75    , 194.875   , 135.25    ],
         [187.625   , 191.75    , 132.90625 ],
         [185.14062 , 189.26562 , 131.17188 ],
         ...,
         [109.71875 , 134.23438 ,  61.546875],
         [107.5625  , 130.67188 ,  56.4375  ],
         [104.75    , 127.      ,  51.125   ]],

        ...,

        [[175.5     , 213.625   , 

In [10]:
# Initialize interpreter
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("\nModel input shape:", input_details[0]['shape'])


Model input shape: [  1 300 300   3]


In [11]:
# Run inference
interpreter.set_tensor(input_details[0]['index'], input_img_arr)
interpreter.invoke()
predictions = interpreter.get_tensor(output_details[0]['index'])

In [13]:
predictions

array([[5.69111489e-06, 1.01257128e-05, 1.09913581e-05, 5.13146142e-06,
        6.46148510e-06, 7.06304054e-05, 6.37469711e-05, 8.04384938e-04,
        3.39219478e-05, 4.14809474e-05, 3.07058435e-05, 4.77442736e-05,
        8.12278711e-04, 2.71316472e-04, 1.05563871e-04, 8.71187694e-06,
        7.09761298e-05, 5.74782207e-05, 4.25061298e-05, 7.59235190e-05,
        3.45572371e-05, 2.08172296e-05, 4.08638851e-04, 7.02968740e-04,
        6.28813577e-05, 5.54926119e-05, 4.15214745e-05, 1.56165741e-04,
        1.23978744e-03, 5.11721300e-05, 6.33126838e-05, 2.02557239e-05,
        7.04237827e-05, 1.70887564e-04, 2.30636055e-04, 8.76866325e-05,
        1.21425837e-05, 5.56365587e-04, 8.86036083e-04, 1.63764646e-03,
        1.01025289e-04, 1.36743256e-04, 1.21325422e-04, 6.09467133e-05,
        5.29334648e-04, 9.45806532e-05, 1.17248630e-04, 1.60235941e-05,
        1.08172841e-04, 2.13108997e-05, 1.13829458e-06, 1.74956604e-05,
        1.30525586e-05, 9.76306255e-05, 1.95559824e-05, 1.075933

In [15]:
import numpy as np

predicted_class = np.argmax(predictions)
confidence = np.max(predictions)

In [17]:
predicted_class, confidence

(np.int64(98), np.float32(0.963154))

In [18]:
print(f"Class: {predicted_class} (Confidence: {confidence:.2%})")

Class: 98 (Confidence: 96.32%)


### Compare prediction of TFlite model with the keras model

In [19]:
# Prediction using keras model
keras_pred = keras_model.predict(input_img_arr)
keras_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


array([[5.77862147e-06, 1.02814065e-05, 1.11385853e-05, 5.21036327e-06,
        6.54803716e-06, 7.16464347e-05, 6.42234445e-05, 8.15956155e-04,
        3.43763350e-05, 4.21187651e-05, 3.11171498e-05, 4.83365839e-05,
        8.25574389e-04, 2.76330305e-04, 1.07344174e-04, 8.84583278e-06,
        7.21378747e-05, 5.81912827e-05, 4.29914471e-05, 7.69405015e-05,
        3.49859511e-05, 2.10960770e-05, 4.13910428e-04, 7.13080983e-04,
        6.35372417e-05, 5.60714143e-05, 4.22423436e-05, 1.57794595e-04,
        1.26377761e-03, 5.17564004e-05, 6.42861924e-05, 2.05270462e-05,
        7.12974434e-05, 1.71745021e-04, 2.34071587e-04, 8.89045841e-05,
        1.23052332e-05, 5.59977023e-04, 8.93530843e-04, 1.64827658e-03,
        1.02228667e-04, 1.38270800e-04, 1.22890546e-04, 6.14622477e-05,
        5.34072926e-04, 9.51481052e-05, 1.19342476e-04, 1.62382312e-05,
        1.09247761e-04, 2.16385797e-05, 1.15129114e-06, 1.76954218e-05,
        1.31758288e-05, 9.88900865e-05, 1.97406680e-05, 1.085564

In [20]:
keras_predicted_class = np.argmax(keras_pred)
keras_confidence = np.max(keras_pred)

print(f"Prediction with keras model \nClass: {keras_predicted_class} (Confidence: {keras_confidence:.2%})")

Prediction with keras model 
Class: 98 (Confidence: 96.28%)


In [21]:
# Check some more values
tflite_pred = predictions

# Print first 5 values
print(f"Keras output: {keras_pred[0][:5]}...")
print(f"TFLite output: {tflite_pred[0][:5]}...")


Keras output: [5.77862147e-06 1.02814065e-05 1.11385853e-05 5.21036327e-06
 6.54803716e-06]...
TFLite output: [5.6911149e-06 1.0125713e-05 1.0991358e-05 5.1314614e-06 6.4614851e-06]...


Almost Same!!

And predicted class from tflite and keras model are same.